In [25]:
import pandas as pd
import requests


In [31]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
web_text=requests.get(url).content

In [27]:
conda install -c anaconda lxml


Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [28]:
conda install -c anaconda beautifulsoup4


Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [45]:
from bs4 import BeautifulSoup

Web scraping and put into DataFrame

In [46]:
soup = BeautifulSoup(web_text, 'html.parser')

In [48]:
table = soup.find('table')

In [50]:
df=pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])

In [52]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [53]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Data Cleaning

In [55]:
# remove rows where Borough is 'Not Assigned'

In [58]:
df = df[~df['Borough'].isnull()]  
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
df['Neighborhood'].replace('Not assigned',df['Borough'],inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [59]:
df.shape

(103, 3)

PART 2 We need to get the latitude and the longitude coordinates of each neighborhood.

In [60]:
coor=pd.read_csv('http://cocl.us/Geospatial_data')
coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [66]:
coor.rename(columns={'Postal Code':'PostalCode'},inplace=True)
result= pd.merge(coor, df, on='PostalCode')
result=result[['PostalCode', 'Borough', 'Neighborhood','Latitude','Longitude']]
result.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
